## CSC 475 / SENG 480B - Assignment 5

#### [YOUR NAME HERE]

**Note:** Please upload your completed notebook as a zip file with all audio files used to complete your assignment. Only include the audio files that you used to keep the upload size as small as possible. (i.e. do not upload all of GTZAN or any other dataset) 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import IPython.display as ipd
from scipy.interpolate import interp1d

### Question 1

For this question use a 30 second clip of audio that you like. You can either use you own or use any of the clips in the GTZAN dataset. Try to select a clip that has a repeating rhythm, phrase, or harmonic structure that you can identify in a self-similarity matrix - you may need to experiment with a few different audio files to find one that works well. You can download GTZAN here: https://drive.google.com/file/d/16qBM8tYXn2z5LOl82mjnRAxQyZsJ0xdP/view?usp=sharing

**1a**

Repeat the 30 second recording to create a one minute recording. Plot the [cross-similarity matrix](https://librosa.org/doc/latest/generated/librosa.segment.cross_similarity.html) for this new one minute long clip with the original 30 second recording.  Describe how the repetition can be seen in a plot of the cross-similarity matrix. Use MFCC features and the affinity mode.

*(Minimum: 1 pt)*

**1b**

Plot the self-similarity matrix for your original 30 second clip (i.e the cross-similarity to itself). Visually identify a repeating structure (it could be a bar, a phrase, a segment) on the self-similarity matrix, describe it, and generate two audio fragments that demonstrate this repetition. Hint: repetition shows as block structure, you will need to map the dimensions of the repeating block to time to select the audio fragments.

*(Minimum: 1 pt)*

**1c**

Use [time stretching](https://librosa.org/doc/latest/generated/librosa.effects.time_stretch.html) on your audio recording to create the following modified signal: the first 10 seconds should be slowed down (rate 0.75), the middle 10 seconds should remain the same, and the last 10 seconds should be sped up (rate 1.25). Plot the cross-similarity matrix between the original and modified recording (use MFCC features and the affinity mode) and describe how the time-stretching can be observed visually. 

*(Expected: 1pt)*

**1d**

Use [harmonic/percussive sound source separation](https://librosa.org/doc/latest/generated/librosa.effects.hpss.html) to generate a percussive track and a harmonic track from your 30 second example. Plot the self-similarity matrices using affinity for the percussive and harmonic versions using MFCCs as well as Chroma (use [chroma_cqt](https://librosa.org/doc/latest/generated/librosa.feature.chroma_cqt.html) or [chroma_stft](https://librosa.org/doc/latest/generated/librosa.feature.chroma_stft.html)). Based on the resulting four plots discuss feature set works better for each configuration (harmonic/percussive).

*(Expected: 1pt)*

**1e**

Use [Dynamic Time Warping](https://librosa.org/doc/latest/generated/librosa.sequence.dtw.htm) using the original and modified (time-stretched) recording you created in the previous subquestion. Plot the cost matrix and associated optimal path and describe how the optimal path reflects the time stretching. Show how you can estimate the time-stretching rates from the optimal path. You can assume that you know that the rate is going to change every 10 seconds but you don’t know what the corresponding rates are. Test your procedure with a set of different time stretching rates.

*(Expected: 1 pt)*

### Question 2

In this question you will be using a state-of-the art deep learning sound source separation library: https://github.com/deezer/spleeter to explore sound source separation, and transcription. For answering the question you can either select a track that you like or use one of the tracks from musdb (a database for sound source separation for music for which spleeter performs quite well): https://sigsep.github.io/datasets/musdb.html.

**2a**

Run the sound source separation with the 4 stem model on your example audio recording. Listen and plot the time-domain waveforms of the 4 individual stems. Comment on what you hear. 

*(Minimum: 1pt)*

**2b**

Pitch shift the vocal stem by a major third (4 semitones) using [librosa.effects.pitch_shift](https://librosa.org/doc/latest/generated/librosa.effects.pitch_shift.html) and mix the pitch shifted audio with the other stems. Listen to the result and comment on it. 

*(Minimum: 1pt)*

**2c**

Run monophonic pitch detection (either your own implementation or using some library like librosa) on the vocal stem, sonify it using a sinusoid (you can use the sonification code provided below) and mix with the drum track. Listen to the result and comment on whether you still recognize the song. 

*(Expected: 1 pt)*

In [2]:
# Function to sonify a pitch track into a sine wave
def sonify(pitch_track, srate, hop_size):

    times = np.arange(0.0, float(hop_size * len(pitch_track)) / srate,
                      float(hop_size) / srate)

    # sample locations in time (seconds)                                                      
    sample_times = np.linspace(0, np.max(times), int(np.max(times)*srate-1))

    # create linear interpolators for frequencies and amplitudes                              
    # so that we have a frequency and amplitude value for 
    # every sample 
    freq_interpolator = interp1d(times,pitch_track)

    # use the interpolators to calculate per sample frequency and                             
    # ampitude values                                                                         
    sample_freqs = freq_interpolator(sample_times)

    # create audio signal                                                                     
    audio = np.zeros(len(sample_times));
    T = 1.0 / srate
    phase = 0.0
    
    # update phase according to the sample frequencies 
    for i in range(1, len(audio)):
        audio[i] =  np.sin(phase)
        phase = phase + (2*np.pi*T*sample_freqs[i])

    return audio

**2d**

Inspect the magnitude spectrogram of the drum track and try to identify features that characterize the kick sound and the snare sound. Based on your visual inspection write a simple kick and snare detection method that outputs the times where a kick or snare drum occurs. Your method only needs to work for the specific example you are examining and does not need to be perfect. Create a new track by placing the kick.wav and snare.wav samples from the resources folder in the detected locations. Mix your synthetic toy drum track and sinusoid sonification from the previous question and listen to the result. Can you still identify the song? 

*(Advanced: 1 pt)*

**2e**

Run beat tracking on the drum track (result from spleeter) and then use the resulting beat onsets combined with the extracted pitch from the vocal melody to transcribe the vocal melody to music notation using music21. For each beat map the median MIDI pitch of the vocals to a tiny notation string. Listen to the generated MIDI melody using Music21. Can you still identify the melody?

*(Advanced: 1 pt)*